# SK Part 2: Feature Selection and Ranking
***

The topic of this tutorial is feature selection and ranking (as in "what are the most important 5 features?"). Feature selection is usually an overlooked issue in machine learning. In many cases, using all the descriptive features that are available can lead to excessive computational times, overfitting, and poor performance in general. It's always a good idea to check to see if we can achieve better performance by employing some sort of feature selection before fitting a model. Feature selection can also be performed in conjunction with the modeling process (as part of a machine learning pipeline), which is discussed in **SK Part 5**.

## Learning Objectives
* Perform feature selection and ranking using the following methods:
  - F-score (a statistical filter method)
  - Mutual information (an entropy-based filter method)
  - Random forest importance (an ensemble-based filter method)
  - spFSR (feature selection using stochastic optimisation)
* Compare performance of feature selection methods using paired t-tests. 

First, let's discuss some terminology. 

The classifier used to assess performance of the feature selection methods is called the "wrapper". Here, we use the decision tree classifier as the wrapper. As for the sample data, we use the breast cancer Wisconsin dataset.

The first three methods are **"filter methods"**: they examine the relationship between the descriptive features and the target feature and they select features only once regardless of which classifier shall be used subsequently.

The last method is a **"wrapper method"**: it selects a different set of features for each wrapper. Wrapper feature selection methods are relatively slow and they need to be executed again when a different wrapper is used. For instance, the best 5 features selected by a wrapper method for a decision tree classifier will probably be different than the best 5 features for the 1-nearest neighbor classifier. However, wrapper methods attempt to solve the "real problem": "what is the best subset of features when, say, the decision tree classifier is used?". They usually perform better than the filter methods at the cost of more computational resources and a different set of features for each classifier.

## Table of Contents
  * [Data Preparation](#Data-Preparation)
  * [Performance with Full Set of Features](#Performance-with-Full-Set-of-Features)
  * [Feature Selection Using F-Score](#Feature-Selection-Using-F-Score)
  * [Feature Selection Using Mutual Information](#Feature-Selection-Using-Mutual-Information)
  * [Feature Selection Using Random Forest Importance](#Feature-Selection-Using-Random-Forest-Importance)
  * [Feature Selection Using spFSR](#Feature-Selection-Using-spFSR)
  * [Performance Comparison Using Paired T-Tests](#Performance-Comparison-Using-Paired-T-Tests)

## Data Preparation

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold
import sklearn.metrics as metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn import feature_selection as fs
from sklearn.tree import DecisionTreeClassifier

Let's load the dataset from the Cloud.

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import io
import requests

# so that we can see all the columns
pd.set_option('display.max_columns', None)
# then upload the breast_cancer_wisconsin.csv data file from your computer
df = pd.read_csv('breast_cancer_wisconsin.csv')
# how to read a csv file from a github account
#url_name = 'https://raw.githubusercontent.com/akmand/datasets/master/breast_cancer_wisconsin.csv'
#url_content = requests.get(url_name, verify=False).content
#df = pd.read_csv(io.StringIO(url_content.decode('utf-8')))

Let's have a look at the first 5 rows to see what the dataset looks like. Here, the last column "diagnosis" is the target variable.

In [ ]:
df.head()

Let's do some pre-processing:
- Split the dataset columns into `Data` and `target`.
- Make target numeric by label-encoding.
- Normalize each descriptive feature in `Data` to be between 0 and 1.

In [ ]:
Data = df.drop(columns = 'diagnosis').values
target = df['diagnosis']
test=df["worst_area"]
#Data = preprocessing.MinMaxScaler().fit_transform(Data)
#target = preprocessing.LabelEncoder().fit_transform(target)

In [ ]:
len(test)
test.head()
test[0]

Data[0,: 1]
len(Data)
Data.shape


In [ ]:
for col in Data.

## Performance with Full Set of Features

As wrapper, we use the decision tree classifier with default values.

In [ ]:
clf = DecisionTreeClassifier(random_state=999)

First, we would like to assess performance using all the features in the dataset. For assessment, we shall use stratified 5-fold cross-validation with 3 repetitions. We set the random state to 999 so that our results can be replicated and verified later on exactly as they are.

In [ ]:
cv_method = RepeatedStratifiedKFold(n_splits=5, 
                                     n_repeats=3,
                                     random_state=999)

For scoring, we use the accuracy score.

In [ ]:
scoring_metric = 'accuracy'

Let's perform the cross-validation using the `cross_val_score` function.

In [ ]:
cv_results_full = cross_val_score(estimator=clf,
                             X=Data,
                             y=target, 
                             cv=cv_method, 
                             scoring=scoring_metric)

The array `cv_results_full` contains 15 values corresponding to each one of the 3-repetition/ 5-fold combinations.

In [ ]:
cv_results_full

We compute the average cross-validation performance as the mean of the `cv_results_full` array.

In [ ]:
cv_results_full.mean().round(3)

Let's now select the best 5 features in the dataset using different methods.

In [ ]:
num_features = 5

## Feature Selection Using F-Score

The F-Score method is a filter feature selection method that looks at the relationship between each descriptive feature and the target feature using the F-distribution. 

The code below returns the indices of the 5 features that have the highest F-Score value sorted from the highest to the lowest. Pay attention that the wrapper is not used in any way when selecting features using the F-Score method.

In some cases, the F-Score will be "NaN" for some features due to technical reasons (related to the nature of the F-distribution). For this reason, we will convert any "NaN" value to zero for a correct result via the `np.nan_to_num()` method. 

In [ ]:
fs_fit_fscore = fs.SelectKBest(fs.f_classif, k=num_features)
fs_fit_fscore.fit_transform(Data, target)
fs_indices_fscore = np.argsort(np.nan_to_num(fs_fit_fscore.scores_))[::-1][0:num_features]
fs_indices_fscore

Let's see what these 5 best features are.

In [ ]:
best_features_fscore = df.columns[fs_indices_fscore].values
best_features_fscore

Based on the F-Scores, we observe that, out of the top 5 features, the most important feature is "worst_concave_points" and the least important feature is "mean_perimeter".

The F-Score importances of these features are given below.

In [ ]:
feature_importances_fscore = fs_fit_fscore.scores_[fs_indices_fscore]
feature_importances_fscore

We define a function for plotting so that we can plot other importance types as well corresponding to different feature selection methods.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline 
%config InlineBackend.figure_format = 'retina'
plt.style.use("ggplot")

def plot_imp(best_features, scores, method_name):   
    plt.barh(best_features, scores)
    plt.title(method_name + ' Feature Importances')
    plt.xlabel("Importance")
    plt.ylabel("Features")
    plt.show()
    

In [ ]:
plot_imp(best_features_fscore, feature_importances_fscore, 'F-Score')

We can select those features from the set of descriptive features `Data` using slicing as shown below.

In [ ]:
Data[:, fs_indices_fscore].shape

Let's now assess performance of this feature selection method using cross validation with the decision tree classifier.

In [ ]:
cv_results_fscore = cross_val_score(estimator=clf,
                             X=Data[:, fs_indices_fscore],
                             y=target, 
                             cv=cv_method, 
                             scoring=scoring_metric)
cv_results_fscore.mean().round(3)

## Feature Selection Using Mutual Information

The mutual information method is a filter feature selection method that looks at the relationship between each descriptive feature and the target feature using the concept of entropy.

The code below returns the indices of the 5 features that have the highest mutual information value. As in the F-score method, the wrapper is not used in any way when selecting features using the mutual information method.

In [ ]:
fs_fit_mutual_info = fs.SelectKBest(fs.mutual_info_classif, k=num_features)
fs_fit_mutual_info.fit_transform(Data, target)
fs_indices_mutual_info = np.argsort(fs_fit_mutual_info.scores_)[::-1][0:num_features]
best_features_mutual_info = df.columns[fs_indices_mutual_info].values
best_features_mutual_info

In [ ]:
feature_importances_mutual_info = fs_fit_mutual_info.scores_[fs_indices_mutual_info]
feature_importances_mutual_info

In [ ]:
plot_imp(best_features_mutual_info, feature_importances_mutual_info, 'Mutual Information')

Now let's evaluate the performance of these features.

In [ ]:
cv_results_mutual_info = cross_val_score(estimator=clf,
                             X=Data[:, fs_indices_mutual_info],
                             y=target, 
                             cv=cv_method, 
                             scoring=scoring_metric)
cv_results_mutual_info.mean().round(3)

## Feature Selection Using Random Forest Importance

The random forest importance (RFI) method is a filter feature selection method that uses the total decrease in node impurities from splitting on a particular feature as averaged over all decision trees in the ensemble. For classification, the node impurity is measured by the Gini index and for regression, it is measured by residual sum of squares.

Let's perform RFI feature selection using 100 trees.

In [ ]:
model_rfi = RandomForestClassifier(n_estimators=100)
model_rfi.fit(Data, target)
fs_indices_rfi = np.argsort(model_rfi.feature_importances_)[::-1][0:num_features]

Here are the best features selected by RFI.

In [ ]:
best_features_rfi = df.columns[fs_indices_rfi].values
best_features_rfi

In [ ]:
feature_importances_rfi = model_rfi.feature_importances_[fs_indices_rfi]
feature_importances_rfi

In [ ]:
plot_imp(best_features_rfi, feature_importances_rfi, 'Random Forest')

Now let's evaluate the performance of these features.

In [ ]:
cv_results_rfi = cross_val_score(estimator=clf,
                             X=Data[:, fs_indices_rfi],
                             y=target, 
                             cv=cv_method, 
                             scoring=scoring_metric)
cv_results_rfi.mean().round(3)

## Feature Selection Using spFSR

spFSR is a new wrapper-based feature selection method that uses binary stochastic approximation. Please refer to this [journal article](https://www.sciencedirect.com/science/article/abs/pii/S0957417422018826) paper for more information on this method.

First, you need to make sure that you download and copy the Python file `spFSR.py` under the same directory as your Jupyter notebook so that the import works correctly.

Let's define an SpFSR object with our feature selection problem with 'accuracy' as our performance metric. Let's run spFSR in the wrapper mode by setting `wrapper=clf`.

Please keep in mind that spFSR can be used as a **filter-based** feature selection method as well so that the selected features do not depend on the intended classifier. For this to work, just set the `wrapper` parameter to `None`. For more details, please see [this](https://github.com/akmand/spFSR/blob/master/spFSR_example_github.py) example.

In [ ]:
# pay attention to capitalization below!
from spFSR import SpFSR

# set the engine parameters
# pred_type needs to be 'c' for classification and 'r' for regression datasets
sp_engine = SpFSR(x=Data, y=target, pred_type='c', wrapper=clf, scoring='accuracy')

Let's now run the spFSR method and the indices of the best features.

In [ ]:
np.random.seed(999)
sp_output = sp_engine.run(num_features=num_features).results

Let's get the indices of the best features.

In [ ]:
fs_indices_spfsr = sp_output.get('selected_features')
fs_indices_spfsr

Let's have a look at the top 5 features selected by spFSR.

In [ ]:
best_features_spfsr = df.columns[fs_indices_spfsr]
best_features_spfsr

In [ ]:
feature_importances_spfsr = sp_output.get('selected_ft_importance')
feature_importances_spfsr

In [ ]:
plot_imp(best_features_spfsr, feature_importances_spfsr, 'spFSR')

Finally, let's evaluate the performance of the spFSR feature selection method.

In [ ]:
cv_results_spfsr = cross_val_score(estimator=clf,
                             X=Data[:, fs_indices_spfsr],
                             y=target, 
                             cv=cv_method, 
                             scoring=scoring_metric)
cv_results_spfsr.mean().round(3)

We observe that we get a cross-validation accuracy of 94.4% with spFSR with just 5 features. Recall that the cross-validation accuracy with the full set of features is 93%. So, in this particular case, it is remarkable that we can achieve even slightly better results with 5 features selected by spFSR as opposed to using all the 30 features. However, we will need to conduct a t-test to determine if this difference is statistically significant or not.

## Performance Comparison Using Paired T-Tests

For performance assessment, we used repeated cross-validation. However, cross-validation is a random process and we need statistical tests in order to determine if any difference between the performance of any two feature selection methods is statistically significant; or if it is within the sample variation and the difference is statistically insignificant.

Since we fixed the random state to be same for all cross-validation procedures, all feature selection methods were fitted and then tested on exactly the same data partitions. This indicates that our experiments were actually paired. Conducting experiments in a paired fashion reduces the variability significantly compared to conducting experiments in an independent fashion.

Let's now conduct paired t-tests to see which differences between full set of features, filter methods, and spFSR are statistically significant. Let's first remind ourselves the performances.

In [ ]:
print('Full Set of Features:', cv_results_full.mean().round(3))
print('F-Score:', cv_results_fscore.mean().round(3))
print('Mutual Information:', cv_results_mutual_info.mean().round(3))
print('RFI:', cv_results_rfi.mean().round(3))
print('spFSR:', cv_results_spfsr.mean().round(3)) 

The above results indicate that spFSR outperforms the other FS methods. However, we need to perform some statistical tests to check to see if this difference is indeed statistically significant.

For a paired t-test in Python, we use the `stats.ttest_rel` function inside the `scipy` module and look at the p-values. At a 95% significance level, if the p-value is smaller than 0.05, we can conclude that the difference is statistically significant.

In [ ]:
from scipy import stats
print(stats.ttest_rel(cv_results_spfsr, cv_results_fscore).pvalue.round(3))
print(stats.ttest_rel(cv_results_spfsr, cv_results_mutual_info).pvalue.round(3))
print(stats.ttest_rel(cv_results_spfsr, cv_results_rfi).pvalue.round(3))

Since all p-values are below 0.05, we conclude that spFSR is statistically better than the other FS methods. Next, let's see if spFSR performance is better than that with full set of features.

In [ ]:
stats.ttest_rel(cv_results_spfsr, cv_results_full).pvalue.round(3)

For spFSR vs. full set of features, we observe a p-value above 0.05, indicating that the difference is not statically significant. Thus, spFSR with 5 features performs statistically the same as the full set of features, at least for the decision tree classifier.

**Note**: In this notebook, we use all the data to train the feature selection methods and then tested them again on the entire dataset using cross-validation due to the smallness of the dataset to work with. Despite its simplicity, this approach potentially results in overfitting. In order to mitigate this issue, a more appropriate approach would be to perform this comparison within a **train/ test split approach**. Specifically, we can split the entire data into two partitions, a train and a test data. We can find the top features using the **train data** using cross-validation. Next, we can assess the performance of these features on the **test data**, again using cross-validation.